# 1. Data Acquisition & Data Preprocessing

**Author**: Navavat Pipatsart, pnastranagant@gmail.com

**language**: python

**environemt**: jupyter notebook

**Objective**: To perform data preprocessing using signal processing approach then save for **Part 2. Exploratory Data Analysis & Data Preparation**

**Last modified date**: 2022-06-16

**Modified issue**: 
- implement vc data
- refactor path_to_black_references

**status**: Done

## Environmental Setup

### 1.1. Miscellaneous Configuration

In [ ]:
# Import global library
import numpy as np

# System configuration
%matplotlib inline

### 1.2. Main Configuration

**NOTE**: This project datasets is located in *on-premise external harddisk*. Hence, path will be set to the located files.

In [ ]:
# --------------------------------------------------------------------------------------------------------
# Define paths
## On-premise paths
parent_path = '/Volumes/phdbackup/backup/coriander_samples_data/' # define parent path to files
path_to_black_references = 'black_references/'
path_to_save = '/Volumes/phdbackup/backup/processed_data/preprocessed_data_20220616/' # define path to save files

## On-cloud path
parent_path_cloud = '/Users/pnastra/pnastranagant@gmail.com - Google Drive/My Drive/' # parent path to files in cloud
path_to_wavelengths = parent_path_cloud + 'phd_codes/metadata/wavelengths.csv' # full path to full spectrum wavelengths profile

# Define children path to files
filenames_sample_dict = {
    '2022_02_15_coriander_vb_d1/' : 'black_reference_laptop',
    '2022_02_22_coriander_vb_d8/' : 'black_reference_laptop',
    '2022_02_22_coriander_vbci_d8/' : 'black_reference_laptop',
    '2022_06_08_coriander_vc_d1': 'black_reference_20220608',
    '2022_06_15_coriander_vc_d8': 'black_reference_20220615',
    '2022_06_15_coriander_vcci_d8': 'black_reference_20220615'
    }

# Define path to black references
filenames_black_references_dict = {
    '2022_02_03_coriander_vaci_d8/' : 'black_reference_laptop/capture/coriander__vaci_d8_s26_2022-02-03_08-57-56.raw',
    '2022_02_15_coriander_vb_d1/' : 'black_reference_laptop/capture/coriander__vaci_d8_s26_2022-02-03_08-57-56.raw',
    '2022_02_22_coriander_vb_d8/' : 'black_reference_laptop/capture/coriander__vaci_d8_s26_2022-02-03_08-57-56.raw',
    '2022_02_22_coriander_vbci_d8/' : 'black_reference_laptop/capture/coriander__vaci_d8_s26_2022-02-03_08-57-56.raw',
    '2022_06_08_coriander_vc_d1': 'black_reference_20220608/capture/sample__blackref__2022-06-08_03-05-50.raw',
    '2022_06_15_coriander_vc_d8': 'black_reference_20220615/capture/coriander___blackref__2022-06-15_02-53-36.raw',
    '2022_06_15_coriander_vcci_d8': 'black_reference_20220615/capture/coriander___blackref__2022-06-15_02-53-36.raw'
    }

# --------------------------------------------------------------------------------------------------------
# Image preprocessing variables
wavelet = True # option to enable wavelet denoising
median = True # option to enable median filtering
triangle = True # option to enable triangle segmentation

# --------------------------------------------------------------------------------------------------------
# Define miscellaneous variables
save = True # Data saving option
save_figure = True # Image saving option

# Preload variables
wavelengths = np.fromfile(file=path_to_wavelengths, sep=',') # load list of actual wavelengths

## 2. Define Functions

### 2.1. Data I/O Functions

In [ ]:
# Define function to loading (image data & black reference)
def load_image(path_to_file: str,
               is_black_ref: bool):
    
    '''
    Function to loading (image data & black reference)
    steps:
    1. Loading raw data from assigned 'path_to_file' -> cast data type as 'int16'. The loaded data has 3 axes i.e.
     1.1. Length-axis: a spatial-domain with time-dependence size depending on time-usage while image acquistion
     1.2. Width-axis: a spatial-domain with fixed size as 640
     1.3. Spectral-axis: a spectral-domain with fixed size as 224 (full spectrum)
    2. Reshape loaded data to dimensionality = (lenght, spectral, width, (length, 224, 640))
    3. Swap axes from dimensionality = (lenght, spectral, width, (length, 224, 640)) => dimensionality = (length, width, spectral, (length, 640, 224)) for human intuition
    4. If the loaded data as black reference option is enabled by assigning 'is_black_ref' = True. Slice only top row of length because it is constant in length-axis
    5. Return output image as tensor/ hyperspectral cube
    '''
    
    # Import libraries
    import numpy as np
    
    print('begin to loading file from', path_to_file)
        
    image = np.fromfile(file=path_to_file, dtype='int16') # Load image data from file
    image = image.reshape(-1,224,640) # Reshape => (lenght, spectral, width, (length, 224, 640))
    image = image.swapaxes(1,2) # Swap axes from (lenght, spectral, width, (length, 224, 640)) => (length, width, spectral, (length, 640, 224)) for human intuition

    print('loading image done \n')
    
    # Black reference checker
    if is_black_ref == True:
        
        image = image[0,...] # Slice black reference data as matrix (640, 224) <- assume black reference is equal in length-axis
    
        print('black reference has been transformed into matrix (width, spectral)')
        print('loading black reference done')
    
    return image

In [ ]:
# Define function to saving prepared dataset to directory
def save_dataset(data,
                 dataname: str,
                 path_to_save: str):
    
    '''
    Function to saving prepared dataset to directory
    Save dataset to assigned 'path_to_save' with naming by current date and assigned name
    '''
    
    # Import libraries
    import numpy as np
    from datetime import date
    
    print('begin to saving data to path: ', path_to_save)
    print('begin to saving ', dataname)
    
    # Save dataset with date timestamp
    np.save(file=(path_to_save + str(date.today()) + '_' + dataname),
            arr=data) 
    
    print('saving ', dataname , 'done')
    
    return None

### 2.2. Utility Functions

In [ ]:
# Define function to perform PSNR calculation between 2 image tensors
def calculate_psnr(original_image_tensor,
                   processed_image_tensor):
    
    '''
    Function to perform PSNR calculation between 2 image tensors
    1. Loop through spectral-axis then calculate PSNR compare to input image matrix
    2. Return output as image matrix and PSNR
    '''
    
    # Import libraries
    import numpy as np
    from skimage import metrics
    
    # Define collector
    psnr_matrix_denoised = []
    
    # Loop through spectral-axis
    for wavelength in range(original_image_tensor.shape[-1]):
    
        # Compute PSNR as an indication of image quality
        temp = metrics.peak_signal_noise_ratio(image_true=original_image_tensor[...,wavelength], 
                                               image_test=processed_image_tensor[...,wavelength], 
                                               data_range=np.amax(original_image_tensor[...,wavelength]) - np.amin(original_image_tensor[...,wavelength]))
        
        psnr_matrix_denoised.append(temp) # Collect to collector
        
    return psnr_matrix_denoised

In [ ]:
# Define function to extract label from file name
def extract_label(filename: str,
                  label_type: str):
    
    '''
    Function to extract label (y) from file name
    for example:
    file name = data__va_d1_s06_2022-01-27_04-27-39.raw
    if 'label_type' = y: output is va_d1
    if 'label_type' = number: output is 06
    '''
    
    # Label_type y checker
    if label_type == 'y':
        
#         label_extracted = filename.split("__")[-1].split("_s")[0] # Assign label -> cut some unneceasary. y should obtain likes 'va_d1'
        label_extracted = filename.split("coriander_")[-1].split("/")[0] # Assign label -> cut some unneceasary. y should obtain likes 'va_d1'
    
    # Label_type number checker
    elif label_type == 'number':
        
        label_extracted = filename.split('_s')[-1].split('_')[0] # Assign label -> cut some unneceasary. number should obtain likes '06'
    
    return label_extracted

In [ ]:
# Define function to randomly select wavelengths from given filenames
def samplier(path_to_files: str,
             sample_size: int):
    
    '''
    Function to randomly select wavelengths from given filenames
    steps:
    1. Construct list of filenames by excluding non-image file
    2. Randomly select certain filenames from filenames list
    3. Extract selected filenames list from full filename
    4. Ascending sort filenames by sample number
    5. Return output as list
    '''
    
    # Import libraries
    import os
    import random
    
    filenames = [filename for filename in os.listdir(path_to_files) if not filename.startswith('.')] # Extract filenames
    sampled_filenames = random.sample(filenames, sample_size) # Sampling index from filenames
    samples = [int(sample.split('_s')[-1].split('_')[0]) for sample in sampled_filenames] # Extract sample numbers
    samples = sorted(samples) # Sort the numbers
    
    print('chosen sample number:', samples)
    
    return samples

In [ ]:
# Define function to calculate averaged reflectance over spatial domain (width & length axises)
def calculate_average(data,
                      axis):
    
    '''
    Function to calculate averaged reflectance over spatial domain (width & length axises) then return output as signle averaged value
    '''
    
    # Import libraries
    import numpy as np
    
    # Calculate average along spectral-axis of sample image data
    data_averaged = np.mean(data, axis=axis)
    
    return data_averaged

In [ ]:
# Define function to print iterations progress (imported from open-source)
def printProgressBar (iteration, total, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█', printEnd = "\r"):
    
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
        printEnd    - Optional  : end character (e.g. "\r", "\r\n") (Str)
    """
    
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    
    print(f'\r{prefix} |{bar}| {percent}% {suffix}', end = printEnd)
    
    # Print New Line on Complete
    if iteration == total: 
        
        print()
        
    return None

In [ ]:
def convert_to_hours(time_usage):
    
    print('to time usage in hours =', time_usage / (60 * 60))
    
    return None

### 2.3. Image Processing Functions

**Calibration function for reflectance of corrected hyperspectral image**

$ R_{c} = \frac{R_{r} - B}{W - B} \times100\%  $,

where:
1. $ R_{r} $: reflectance of the raw hyperspectral image
2. B: dark current image (~0% reflectance)
3. W: white reference image (~99.9% reflectance)

In [ ]:
# Define function to perform calibration of an input image
def image_calibrater(image_tensor,
                     black_ref_matrix):
    
    '''
    function to perform calibration of an input image
    steps:
    1. find white reference by assuming as the maximum reflectance wavelength-wised of the assigned tensor
    2. calculate raw reflectance (R) - black reference (B) whole image wavelength-wised of the assigned tensor
    3. calculate white reference (W) - black reference (B) top length-axis wavelength-wised of the assigned tensor
    4. calculate calibration by performing (R-B)/(W-B)
    5. return output as tensor
    '''
    
    # Import libraries
    import numpy as np
    
    print('begin to perform calibration')
    
    # White reference preparation
    white_ref_vector = np.amax(image_tensor, axis= (0,1)) # Max argument as vector (224,) along spectral axis -> assign to being white reference
    
    # Calibration preparation
    r_minus_b = np.subtract(image_tensor, black_ref_matrix) # = reflectance - black reference (length, 640, 224)
    w_minus_b = np.subtract(white_ref_vector, black_ref_matrix) # = white reference - black reference (640, 224)
    image_tensor_calibrated = np.divide(r_minus_b, w_minus_b[np.newaxis, :, :]) # Perform calibration
    
    print('calibration done')
    
    return image_tensor_calibrated

In [ ]:
# Define function to perform image resizing image data with specific size using interpolation
def resize_image(image_tensor,
                  output_size: int):
    
    '''
    Function to perform image resizing image data with specific size using interpolation as return outout as image matrix
    '''
    
    # Import libraries
    import numpy as np
    from skimage import transform
    
    print('begin to perform local mean resizing')
    
    # Assign collector
    image_tensor_resized = []
    
    # Loop through spectral-axis
    for wavelength in range(image_tensor.shape[-1]):
    
        # Perform resizing image
        temp = transform.resize_local_mean(image=image_tensor[...,wavelength], 
                                           output_shape=(output_size, output_size))
        
        image_tensor_resized.append(temp) # Collect to collector
        
    image_tensor_resized = np.stack(image_tensor_resized, axis=-1) # Reformat list => array
    
    print('local mean resizing done')
    
    return image_tensor_resized

In [ ]:
# Define function to perform periodic-noise reduction in frequency-domain of the image using difference of Gaussian algorithm
def filter_gaussians(image_matrix):
    
    # Import libraries
    import numpy as np
    from skimage import filters
    from skimage import metrics
    
    '''
    Function to perform periodic-noise reduction image filtering in frequency-domain using difference of Gaussian algorithm
    1. Filter image using difference of Gaussians algorithm
    2. Improve ROI using window image
    3. Calculate PSNR compare to input image matrix
    4. Return output as image matrix and PSNR
    '''
    
    # Filter image
    image_pre_filtered = filters.difference_of_gaussians(image=image_matrix, 
                                                         low_sigma=2, 
                                                         high_sigma=50)
    
    image_filtered = image_pre_filtered * filters.window('hann', image_matrix.shape) # Window image to improve FFT

    # Compute PSNR as an indication of image quality
    psnr_filtered = metrics.peak_signal_noise_ratio(image_matrix, image_filtered, data_range=np.amax(image_matrix)-np.amin(image_matrix))
    
    return image_filtered, psnr_filtered

In [ ]:
# Define function to perform wavelet denoising on an image using BayesShrink algorithm with Daubechies wavelet
def denoiser_wavelet(image_tensor):
    
    # Import libraries
    import numpy as np
    from skimage import restoration
    
    '''
    Function to perform wavelet denoising on an image using BayesShrink algorithm with Daubechies wavelet
    1. Denoise image using wavelet denoising
    2. Calculate PSNR compare to input image matrix
    3. Return output as image matrix and PSNR
    '''
    
    print('begin to perform wavelet denoising')
    
    # Denoising image
    image_tensor_denoised = restoration.denoise_wavelet(image=image_tensor,
                                                         wavelet='db2', 
                                                         mode='soft', 
                                                         method='BayesShrink',
                                                         channel_axis=-1)
    
    # Calculate PSNR
    psnr_matrix_denoised = calculate_psnr(original_image_tensor=image_tensor,
                                          processed_image_tensor=image_tensor_denoised)
    
    print('wavelet denoising done')

    return image_tensor_denoised, psnr_matrix_denoised

In [ ]:
# Define function to perform Gaussian, random, and salt and pepper-noise reduction image filtering using non-linear median algorithm
def filter_median(image_tensor):
    
    # Import libraries
    import numpy as np
    from skimage import filters
    from skimage import morphology
    
    '''
    Function to perform Gaussian, random, and salt and pepper-noise reduction image filtering using non-linear median algorithm
    1. Define kernel/footprint for filering
    2. Filter image using non-linear median algorithm
    3. Calculate PSNR compare to input image matrix
    4. Return output as image matrix and PSNR
    '''
    
    print('begin to perform median filtering')
    
    kernel_size = 3
    footprint = np.ones((kernel_size, kernel_size, image_tensor.shape[-1]))
    
    # Filter image
    image_tensor_filtered = filters.median(image=image_tensor,
                                           footprint=footprint)

    # Calculate PSNR
    psnr_matrix_filtered = calculate_psnr(original_image_tensor=image_tensor,
                                          processed_image_tensor=image_tensor_filtered)
    
    print('median filtering done')
    
    return image_tensor_filtered, psnr_matrix_filtered

In [ ]:
# Define function to perform calculate threshold value using triangle algorithm then construct mask and apply to image 
def thresholder_triangle(image_tensor):
    
    '''
    Function to perform calculate threshold value using triangle algorithm then construct mask and apply to image 
    1. Calculate threshold value using triangle algorithm for lesser noise-spectrum image -> assume as representator
    2. Construct boolean mask
    3. Loop through spectral-domain, create zeros-array shape like image matrix
    4. Apply mask to image matrix
    5. Collect to collector and reformat collector from list => numpy array
    6. Calculate PSNR
    7. Return output
    '''
    
    # Import libraries
    import numpy as np
    from skimage import filters
        
    print('begin to perform triangle thresholding')
    
    threshold_value = filters.threshold_triangle(image=image_tensor[...,50]) # Calculate threshold value using triangle algorithm
    mask = image_tensor[...,50] > threshold_value # Construct boolean mask

    # Assign collector
    image_tensor_thresholded = []
    
    # Loop through spectral-domain
    for wavelength in range(image_tensor.shape[-1]):
        
        temp = np.zeros_like(image_tensor[..., wavelength]) # Create zeros-array shape like image matrix
        temp[mask] = image_tensor[..., wavelength][mask] # Apply mask to image matrix
        image_tensor_thresholded.append(temp) # Collect to collector
       
    image_tensor_thresholded = np.stack(arrays=image_tensor_thresholded, axis=-1) # Collect to collector
    
    # Calculate PSNR
    psnr_matrix_thresholded = calculate_psnr(original_image_tensor=image_tensor,
                                             processed_image_tensor=image_tensor_thresholded)
    
    print('triangle thresholding done')
    
    return image_tensor_thresholded, psnr_matrix_thresholded

### 2.4. Visualization Functions

In [ ]:
# Define function to comparative visualize between 2 images
def viz_compare(image_1,
                image_2,
                title_1: str,
                title_2: str,
                save: bool,
                figurename: str,
                path_to_save: str):
    
    '''
    Dunction to comparative visualize between 2 images
    steps:
    1. Create empty figure for drawing sub-figures of images
    2. Draw image 1
    3. Draw image 2
    4. Consider save figure option is enabled by 'save' = True. Save figure to assigned 'path_to_save' with naming by current date and assigned name
    '''
    
    # Import libraries
    from datetime import date
    from matplotlib import pyplot as plt # Figure drawing
    from scipy import ndimage as ndimage # Image rotation
    
    # Figure configuration
    fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(10, 8), sharex=True, sharey=True)
    fontsize = 16
    fig.suptitle(figurename, fontsize=fontsize)

    # Draw 1st image
    ax1.imshow(ndimage.rotate(input=image_1, angle=180))
    ax1.set_title(title_1, fontsize=fontsize) # Set image title
    ax1.axis("off") # Disable axes label

     # Draw 2nd image
    ax2.imshow(ndimage.rotate(input=image_2, angle=180))
    ax2.set_title(title_2, fontsize=fontsize) # Set image title
    ax2.axis("off") # Disable axes label
    
    # Decoration
    plt.margins(0, 0)
    fig.tight_layout()
    
    # Save figure option checker
    if save == True:
    
        plt.savefig(path_to_save + figurename + '_' + str(date.today()) + '.png') # Save figure
        
        print('save figure:', figurename, 'done') 
        
    plt.show()
    
    return None

In [ ]:
# Define function to comparative visualize among spectral-axis
def viz_spectral(data,
                 indices_list: list,
                 wavelengths: list,
                 save: bool,
                 figurename: str, 
                 path_to_save: str):
    
    '''
    Function to comparative visualize among spectral-axis
    steps:
    1. Loop through assigned wavelengt for drawing 9 images, then draw image
    2. Consider save figure option checker; if save == True: figure
    '''
    
    # Import libraries
    import numpy as np
    from datetime import date
    from matplotlib import pyplot as plt # Figure drawing
    from scipy import ndimage as ndimage # Image rotation
    
    # Figure configuratio
    fig, axs = plt.subplots(nrows=3, ncols=3, figsize=(10, 8), sharex=True, sharey=True)
    fontsize = '16' # Define fontsize
    fig.suptitle(figurename, fontsize=fontsize)
    axs = axs.ravel()

    # Loop through assigned wavelengt for drawing 9 images
    for index, wavelength in enumerate(indices_list):
        
        # Draw image
        axs[index].imshow(ndimage.rotate(input=data[...,wavelength], angle=180), aspect='auto') # draw image
        axs[index].set_title('\u03BB = ' + str(wavelengths[wavelength]) + ' nm', fontsize=fontsize) # Define image title
        axs[index].axis("off") # Disable axes
    
    # Decoration
    fig.tight_layout()
    
    # Save figure option checker
    if save == True:
    
        plt.savefig(path_to_save + figurename + '_' + str(date.today()) + '.png') # save figure
        
        print('save figure:', figurename, 'done')        
        
    plt.show()
    
    return None

### 2.5. Main Function

In [ ]:
# Define main function to execute pipeline of defined functions
def main(parent_path=parent_path,
         path_to_black_references=path_to_black_references,
         path_to_save=path_to_save,
         parent_path_cloud=parent_path_cloud,
         wavelengths=wavelengths,
         filenames_sample_dict=filenames_sample_dict,
         filenames_black_references_dict=filenames_black_references_dict,
         save=save,
         wavelet=wavelet,
         median=median,
         triangle=triangle
         ):
    
    """
    Main function to execute pipeline of defined functions
    steps:
    1. Define necessary metadata
    2. Define collectors and load black reference
    3. Extract sample number
    4. Perform load sample data, calibrate, and calculate averaged refrectance of sample data
    5. Visualize calibrated sample data
    6. Perform data preprocessing using image processing approach, collect PSNR ,and visualize preprocessed sample data
    7. Collect to collectors
    8. Save preprocessed data
    """
    
    # Import libraries
    import os
    import time
    import random
    import numpy as np
    
    start_time_total = time.time() # Memory starting time
    
    # Define list of wavelenght indices to comparative visualization
    indices_list = np.arange(0, len(wavelengths), len(wavelengths) // 9)
    indices_list = np.delete(indices_list, len(indices_list) // 2)
    
    # --------------------------------------------------------------------------------------------------------
    # 1. Define necessary metadata
    # Loop through files for define metadata of each treatment
    for folder in filenames_sample_dict.keys():

        print('begin to execute main function to', folder)

        path_to_files = parent_path + folder # Concatenate path name
        
        # Execute function to extract label from file name #!
        extracted_y = extract_label(filename=folder,
                                    label_type='y')

        print('Dataname is: ', extracted_y)

        # Flip option checker
        if folder in ['2022_06_08_coriander_vc_d1', '2022_06_15_coriander_vc_d8', '2022_06_15_coriander_vcci_d8']:
            
            flip = True # Enable vertically flip to images
            
        else:
            
            flip = False # Disable vertically flip to images
            
        # --------------------------------------------------------------------------------------------------------
        # 2. Define collectors and load black reference
        
        print('begin to loading and preprocessing')
        print('begin to loading black reference')

        start_time = time.time() # Memory starting time

        # Assign global collectors
        X = [] # Image variable data collector
        X_wavelet = []
        X_median = []
        X_triangle = []
        psnr_tensor_wavelet = []
        psnr_tensor_median = []
        psnr_tensor_triangle = []
        y = [] # Image label/output collector
        reflectances_averaged = [] # Averaged reflectance collector
        number = 0 # Counter number of data collector
        
        path_to_black_reference = parent_path + path_to_black_references + filenames_black_references_dict[folder]
        print('path_to_black_reference ===>', path_to_black_reference) #!
        
        # Execute function to loading black reference
        black_ref_matrix = load_image(path_to_file=path_to_black_reference,
                                      is_black_ref=True)
                
        print('begin to loading raw sample data')

        # Iterate through directories
        for subdir, dirs, files in os.walk(path_to_files):

            # Interate through files "inside path to files"
            for file in files:

                # .raw format checker
                if file.endswith('.raw'):

                    # --------------------------------------------------------------------------------------------------------
                    # 3. Extract sample number
                    
                    # Execute function to extract number from file name
                    sample_number = extract_label(filename=file,
                                                  label_type='number')

                    print('sample number: ', sample_number)
                    
                    # --------------------------------------------------------------------------------------------------------
                    # 4. Perform load sample data, calibrate, and calculate averaged refrectance of sample data

                    # Execute function to loading image data
                    image_tensor = load_image(path_to_file=os.path.join(subdir, file),
                                              is_black_ref=False)
                    
                    # Vertically flip option checker
                    if flip == True:
                        
                        image_tensor = np.flip(image_tensor, axis=0) # Vertically flip images
                    
                    # Slice image tensor to preventing "label tag" to compete "calibration bar" in calibration
                    image_tensor = image_tensor[500:, ...]                   

                    # Execute function to perform calibration of an input image
                    image_tensor_calibrated = image_calibrater(image_tensor=image_tensor,
                                                               black_ref_matrix=black_ref_matrix)
                    
                    # Execute function to calculate averaged reflectance over spatial domain (width & length axises)
                    reflectance_averaged = calculate_average(data=image_tensor_calibrated,
                                                                   axis=(0,1))
            
                    # --------------------------------------------------------------------------------------------------------
                    # 5. Visualize calibrated sample data
                    
                    # Execute function to comparative visualize among spectral-axis
                    figurename='Calibrated images of ' + extracted_y + '_' + sample_number

                    viz_spectral(data=image_tensor_calibrated,
                                 indices_list=indices_list,
                                 wavelengths=wavelengths,
                                 save=save_figure,
                                 figurename=figurename, 
                                 path_to_save=path_to_save)

                    # --------------------------------------------------------------------------------------------------------
                    # 6. Perform data preprocessing using image processing approach, collect PSNR ,and visualize preprocessed sample data
                    
                    # Execute function to perform image resizing image data with specific size using interpolation
                    image_tensor_preprocessed = resize_image(image_tensor=image_tensor_calibrated,
                                                             output_size=128)

                    # Execute function to comparative visualize among spectral-axis
                    figurename='Resized images of ' + extracted_y + '_' + sample_number

                    viz_spectral(data=image_tensor_preprocessed,
                                 indices_list=indices_list,
                                 wavelengths=wavelengths,
                                 save=save_figure,
                                 figurename=figurename, 
                                 path_to_save=path_to_save)

                    del image_tensor # Remove unused raw image data for saving memory
                    del image_tensor_calibrated # Remove unused raw image data for saving memory

                    print('begin to perform image processing')

                    # Option to enable wavelet denoising checker
                    if wavelet == True:
                    
                        # Execute function to perform wavelet denoising on an image using BayesShrink algorithm with Daubechies wavelet
                        image_tensor_preprocessed, psnr_matrix_wavelet = denoiser_wavelet(image_tensor=image_tensor_preprocessed)
                
                        X_wavelet.append(image_tensor_preprocessed) # Collect preprocessed data to collector  
                        psnr_tensor_wavelet.append(psnr_matrix_wavelet) # Collect PSNR to collector  

                        # Execute function to comparative visualize among spectral-axis
                        figurename='Wavelet denoised images of ' + extracted_y + '_' + sample_number                        
                        
                        viz_spectral(data=image_tensor_preprocessed,
                                     indices_list=indices_list,
                                     wavelengths=wavelengths, 
                                     figurename=figurename,
                                     save=save_figure,
                                     path_to_save=path_to_save)
                    
                    # Option to enable median filtering checker
                    if median == True:
                        
                        # Execute function to perform Gaussian, random, and salt and pepper-noise reduction image filtering using non-linear median algorithm
                        image_tensor_preprocessed, psnr_matrix_median = filter_median(image_tensor=image_tensor_preprocessed)

                        X_median.append(image_tensor_preprocessed) # Collect preprocessed data to collector
                        psnr_tensor_median.append(psnr_matrix_median) # Collect PSNR to collector
                        
                    # Execute function to comparative visualize among spectral-axis
                    figurename='Median filtered images of ' + extracted_y + '_' + sample_number                        

                    viz_spectral(data=image_tensor_preprocessed,
                                 indices_list=indices_list,
                                 wavelengths=wavelengths, 
                                 figurename=figurename,
                                 save=save_figure,
                                 path_to_save=path_to_save)                        
                        
                    # Option to enable triangle segmentation checker
                    if triangle == True :
                        
                        # Execute function to perform calculate threshold value using triangle algorithm then construct mask and apply to image
                        image_tensor_preprocessed, psnr_matrix_triangle = thresholder_triangle(image_tensor=image_tensor_preprocessed)

                        X_triangle.append(image_tensor_preprocessed) # Collect preprocessed data to collector
                        psnr_tensor_triangle.append(psnr_matrix_triangle) # Collect PSNR to collector

                        # Execute function to comparative visualize among spectral-axis
                        figurename='Triangle threshold images of ' + extracted_y + '_' + sample_number                        
                        
                        viz_spectral(data=image_tensor_preprocessed,
                                     indices_list=indices_list,
                                     wavelengths=wavelengths, 
                                     figurename=figurename,
                                     save=save_figure,
                                     path_to_save=path_to_save)                        
                        
                    # --------------------------------------------------------------------------------------------------------
                    # 7. Collect to collectors
                    # Collect to global collectors
                    y.append(extracted_y) # convert to array is not required
                    reflectances_averaged.append(reflectance_averaged)
                    number += 1 # iterate number of loaded data

                    # Show progress bar
                    printProgressBar(1, len(os.listdir(path_to_files)), prefix = 'Progress:', suffix = 'Complete', length = 50)    

                    print('------------------------------------------------------------------------------------------ \n')

        # Reformat global collectors
        X_wavelet = np.stack(X_wavelet, axis=0)
        X_median = np.stack(X_median, axis=0)
        X_triangle = np.stack(X_triangle, axis=0)
        
        reflectances_averaged = np.concatenate(reflectances_averaged, axis=0) # Transform list => numpy ndarray type
        reflectances_averaged = reflectances_averaged.reshape(-1, 224) # Reshape to (number of samples, 224)

        # Option to enable wavelet denoising checker
        if wavelet == True:
            
            psnr_tensor_wavelet = np.stack(psnr_tensor_wavelet, axis=0) # Convert list => numpy ndarray type
        
        # Option to enable median filtering checker
        if median == True:
                
            psnr_tensor_median = np.stack(psnr_tensor_median, axis=0) # Convert list => numpy ndarray type
         
        # Option to enable triangle segmentation checker
        if triangle == True :
            
            psnr_tensor_triangle = np.stack(psnr_tensor_triangle, axis=0) # Convert list => numpy ndarray type
        
        # --------------------------------------------------------------------------------------------------------
        # 8. Save preprocessed data
        # Save option checker
        if save == True: 
            
            # Execute function to saving processed data
            save_dataset(data=y,
                         dataname='y_' + extracted_y,
                         path_to_save=path_to_save)
            
            save_dataset(data=reflectances_averaged,
                         dataname='reflectances_averaged_' + extracted_y,
                         path_to_save=path_to_save)

            # Option to enable wavelet denoising checker
            if wavelet == True:
            
                save_dataset(data=X_wavelet,
                             dataname='X_wavelet_' + extracted_y,
                             path_to_save=path_to_save)
            
                save_dataset(data=psnr_tensor_wavelet,
                             dataname='psnr_tensor_wavelet_' + extracted_y,
                             path_to_save=path_to_save)

            # Option to enable median filtering checker
            if median == True:

                save_dataset(data=X_median,
                             dataname='X_median_' + extracted_y,
                             path_to_save=path_to_save)
                
                save_dataset(data=psnr_tensor_median,
                             dataname='psnr_median_' + extracted_y,
                             path_to_save=path_to_save)

            # Option to enable triangle segmentation checker
            if triangle == True :

                save_dataset(data=X_triangle,
                             dataname='X_triangle_' + extracted_y,
                             path_to_save=path_to_save)                
                
                save_dataset(data=psnr_tensor_triangle,
                             dataname='psnr_tensor_triangle_' + extracted_y,
                             path_to_save=path_to_save)

        end_time = time.time() # Memory ending time

        print('\n')
        print('loading and preprocessing data from', path_to_files,'done')
        print('summary:')
        print('data has been saved to', path_to_save )
        print(number, 'files have been processed this time')
        print('execution time:', end_time - start_time, 'seconds \n') # calculate time usage
    
    print('-------------------------------------------------------------------------------------------------------- \n')

    end_time_total = time.time() # Memory ending time
    time_usage_total = end_time_total - start_time_total # Calculate time usage
    
    print('\n')
    print('total_execution time:', time_usage_total, 'seconds \n')
    
    convert_to_hours(time_usage=time_usage_total) # Convert time usage to hour unit
    
    return None

## Execute main function

In [ ]:
main()